In [2]:
import mitiq 
import qiskit

In [3]:
from mitiq import *

In [4]:
from qiskit import *

In [5]:
mitiq.SUPPORTED_PROGRAM_TYPES.keys()

['braket', 'cirq', 'pennylane', 'pyquil', 'qibo', 'qiskit']

In [6]:
frontend = "qiskit"

Problem Setup

In [8]:
from mitiq import benchmarks
qc1 = benchmarks.generate_rb_circuits(n_qubits = 2, num_cliffords=7, return_type = frontend)[0]
print(qc1)

     ┌──────────┐┌────┐┌──────────┐ ┌────┐ ┌─────────┐   ┌──────────┐   »
q_0: ┤ Ry(-π/2) ├┤ √X ├┤ Ry(-π/2) ├─┤ √X ├─┤ Ry(π/2) ├─■─┤ Ry(-π/2) ├─■─»
     ├──────────┤├────┤├──────────┤┌┴────┴┐└┬───────┬┘ │ ├─────────┬┘ │ »
q_1: ┤ Ry(-π/2) ├┤ √X ├┤ Ry(-π/2) ├┤ √Xdg ├─┤ Ry(0) ├──■─┤ Ry(π/2) ├──■─»
     └──────────┘└────┘└──────────┘└──────┘ └───────┘    └─────────┘    »
«     ┌─────────┐    ┌─────────┐ ┌───────┐                      ┌─────────┐»
«q_0: ┤ Ry(π/2) ├──■─┤ Ry(π/2) ├─┤ Rx(0) ├────────────────────■─┤ Ry(π/2) ├»
«     ├─────────┴┐ │ ├─────────┤┌┴───────┴─┐┌────┐┌─────────┐ │ └──┬───┬──┘»
«q_1: ┤ Ry(-π/2) ├─■─┤ Ry(π/2) ├┤ Ry(-π/2) ├┤ √X ├┤ Ry(π/2) ├─■────┤ Y ├───»
«     └──────────┘   └─────────┘└──────────┘└────┘└─────────┘      └───┘   »
«     ┌────┐┌───┐ ┌──────┐   ┌─────────┐┌────┐   ┌───┐    ┌───┐     ┌─────────┐»
«q_0: ┤ √X ├┤ Y ├─┤ √Xdg ├─■─┤ Ry(π/2) ├┤ √X ├───┤ Y ├────┤ X ├───■─┤ Ry(π/2) ├»
«     ├────┤├───┤┌┴──────┤ │ └──┬───┬──┘├────┤┌──┴───┴──┐┌┴───┴─┐ │ └─┬──────┬┘»
«q

generate_rb_circuits can only have max 2 qubits

Now we crate an exacutor

In [28]:
from mitiq.interface import convert_to_mitiq
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise

In [31]:
import numpy as np
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit_aer.noise import NoiseModel,errors

# Simulate the circuit with noise
noise_model = initialized_depolarizing_noise(noise_level=0.02)
# Default to a simulator
backend = QasmSimulator(noise_model=noise_model)


def ibmq_executor(circuit: qiskit.QuantumCircuit, shots: int = 8192) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run.
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        basis_gates=noise_model.basis_gates if noise_model else None,
        optimization_level=0, # Important to preserve folded gates.
    )
    
    # Run the circuit
    job = backend.run(exec_circuit, shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result().get_counts()
    if counts.get("0") is None:
        expectation_value = 0.
    else:
        expectation_value = counts.get("0") / shots
    return expectation_value  



In [25]:
# Compute the expectation value of the |0><0| observable.
noisy_value = execute(qc1)
ideal_value = execute(qc1, noise_level1=0.0, noise_level2=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}")

AttributeError: 'Circuit' object has no attribute 'name'